In [1]:
import pandas as pd
import helpers as hlp

In [2]:
# Emis Ef
df_emis = hlp.emis_load_ef()

In [3]:
# load Ofenstatistik
path_ofen = r'\\szh.loc\ugz\Applikationen\Emissionskataster\Dokumentation\2022_Dokumentation\03_Industrie_Gewerbe\c3414_Krematorium\2022 Auswertung Ofenstatistik.xlsx'
df_ofen = pd.read_excel(path_ofen)
df_ofen

,Unnamed: 0,Ofen 1,Ofen 2,Ofen 3,Ofen 4,Ofen 5,Ofen 6,Ofen 7,Total
0,Einäscherungen,1409,416,0,1385,1263,1444,1308,7225
1,Anzahl Kremationen ohne Abbgasreinigung,28,1,0,31,5,38,25,128


In [23]:
einaesch_mit = df_ofen.loc[df_ofen['Unnamed: 0'] == 'Einäscherungen','Total'].values[0]
einaesch_ohne = df_ofen.loc[df_ofen['Unnamed: 0'] == 'Anzahl Kremationen ohne Abbgasreinigung','Total'].values[0]
einaesch_total = einaesch_mit + einaesch_ohne

In [25]:
einaesch_ohne

128

In [11]:
# Emissionsfaktoren aus Messung laden
path = r'\\szh.loc\ugz\Applikationen\Emissionskataster\Dokumentation\2022_Dokumentation\00_Emissionsfaktoren\Emissionsfaktoren_Krematorium_UGZ.xlsx'
ef_krem = pd.read_excel(path)

In [17]:
# Emissionen Krematorium
def krematorium (df_emis, year):

    # EF Erhebung Heinz Jenal 2010 - 2013 [g]
    ef_pm10ex_mit = ef_krem['Feststoffe mit ARE [g/Einaescherung]'].values[0]
    ef_pm10ex_ohne = ef_krem['Feststoffe ohne ARE [g/Einaescherung]'].values[0] 
    ef_pm25ex_mit = ef_pm10ex_mit #Analog EMIS
    ef_pm25ex_ohne = ef_pm10ex_ohne #Analgo EMIS
    ef_voc_mit = ef_krem['VOC mit ARE [g/Einaescherung]'].values[0]
    ef_voc_ohne = ef_krem['VOC ohne ARE [g/Einaescherung]'].values[0]
    ef_co = ef_krem['CO [g/Einäscherung]'].values[0]
    ef_co2 = ef_krem['CO2 [g/Einäscherung]'].values[0]
    ef_so2 = ef_krem['SO2 [g/Einäscherung]'].values[0]
    ef_nox = ef_krem['NOx [g/Einäscherung]'].values[0]
    
    #EF BCex mit ARE hergeleitet aus Verhältnis der BAFU EF BC/PM10ex [g]
    ef_bc_mit = (df_emis[year].loc[df_emis['Prozess'].isin(['Krematorien']) & df_emis['Schadstoff'].isin(['BC exhaust'])].item()) / \
                (df_emis[year].loc[df_emis['Prozess'].isin(['Krematorien']) & df_emis['Schadstoff'].isin(['PM10 exhaust'])].item()) * \
                ef_krem['Feststoffe mit ARE [g/Einaescherung]'].item()
    #EF BCex ohne ARE hergeleitet aus Verhältnis der BAFU EF BC/PM10ex
    ef_bc_ohne = (df_emis[year].loc[df_emis['Prozess'].isin(['Krematorien']) & df_emis['Schadstoff'].isin(['BC exhaust'])].item()) / \
                (df_emis[year].loc[df_emis['Prozess'].isin(['Krematorien']) & df_emis['Schadstoff'].isin(['PM10 exhaust'])].item()) * \
                ef_krem['Feststoffe ohne ARE [g/Einaescherung]'].item()   

    # Emissionsberechnung [kg]
    data = {
            'X': 2682033,
            'Y': 1251106,
            'Emission_PM10ex': (einaesch_mit * ef_pm10ex_mit + einaesch_ohne * ef_pm10ex_ohne) /1000,
            'Emission_NOx':einaesch_total * ef_nox /1000,
            'Emission_VOC': (einaesch_mit * ef_voc_mit + einaesch_ohne * ef_voc_ohne) /1000,
            'Emission_SO2': einaesch_total * ef_so2 /1000,
            'Emission_CO2': einaesch_total * ef_co2 /1000,
            'Emission_CO': einaesch_total * ef_co /1000,
            'Emission_PM25ex': (einaesch_mit * ef_pm25ex_mit + einaesch_ohne * ef_pm25ex_ohne) /1000,
            'Emission_BC': (einaesch_mit * ef_bc_mit + einaesch_ohne * ef_bc_ohne) / 1000
    }

    # Dataframe
    df_emissionen = pd.DataFrame(data, index=[0])

    return df_emissionen

In [18]:
df_emissionen = krematorium(df_emis, 2022)
df_emissionen

,X,Y,Emission_PM10ex,Emission_NOx,Emission_VOC,Emission_SO2,Emission_CO2,Emission_CO,Emission_PM25ex,Emission_BC
0,2682033,1251106,39.641937,1674.602177,14.431957,255.087825,456162.690667,112.528406,39.641937,0.356777


In [44]:
df_emissionen.to_excel(r'Z:\Inputdateien\3_IndustrieGewerbe\c3414_Krematorium_Emissionsmengen.xlsx',sheet_name = 'Krematorium' , index=False)